# Analysis of Vodafone users' fluxes

The study of the flux of people inside urban areas is of paramount importance to achieve an optimal understanding of emerging critical issues in the local mobility, and to explore areas of potential improvements in the infrastructures and local transports.

The mobility of users within and toward Padova has been monitored using the data provided by the Vodafone mobile carrier, which provides the information based on the users' connections to the network cells.
The data provided by the carrier encompasses the monitoring of the users connected to the Vodafone network in Padova in a four-month period from February to May of 2018.

To provide statistical insights on the number and the flow of users, the data is aggregated based on the origin and movements of the users by averaging the number of connections during the time of the monitoring.

To further avoid privacy violation issues, all observations with less than 30 units (e.g. day-areas for which $<$ 30 users have contributed) have been discarded and/or merged into dedicated categories (indicated with "altro", or "other").


## Datasets 

The data is provided in `.csv` files.

* __day_od.csv__: table of the origins and destinations of the users averaged by the day of the week. The data is provided with details of the month, type of user (resident in Padova/Italian visitor/foreign visitor), country of provenance, together with the province and comune of the user (if available).
* __distinct_users_day.csv__: table of the number of distinct users by origin. The data is provided with details of the month, type of user (resident in Padova/Italian visitor/foreign visitor), country of provenance, together with the province and comune of the user (if available).

The information is stored in the fields according to the following scheme: 

- __MONTH__: month analyzed
- __DOW__: day analyzed
- __ORIGIN__: users' origin area (do not consider this field)
- __DESTINATION__: users' destination area (do not consider this field)
- __CUST_CLASS__: user type (resident / Italian visitor / foreigner visitor)
- __COD_COUNTRY__: users' country code (e.g. 222=Italy)
- __COD_PRO__: users' province code (e.g. 12=Varese) 
- __PRO_COM__: users' comune code (e.g. 12026=Busto Arsizio)
- __FLOW__: number of movements for given date-time (with a minimum of 30 users)
- __VISITORS__: overall number of users 

Together with the data files, three lookup-tables are provided to allow matching the Italian institute of STATistics (ISTAT) country, province and comune codes to the actual names.

* __codici_istat_comune.csv__: lookup file containing the mapping between _comune_ ISTAT code-names
* __codici_istat_provincia.csv__: lookup file containing the mapping between _province_ ISTAT code-names
* __codici_nazioni.csv__: lookup file containing mapping the _country_ code to its name

Additional information, useful for the study of the flow of users, as the number of inhabitants of each province and the distance between Padova and all other Italian provinces can be extracted based on the data collected by the ISTAT:

   - English: https://www.istat.it/en/analysis-and-products/databases, Italian: https://www.istat.it/it/dati-analisi-e-prodotti/banche-dati
   
   - English/Italian: https://www.istat.it/en/archive/157423, Italian: https://www.istat.it/it/archivio/157423
   
   - `.zip` package containing the distances between comuni in Veneto region: http://www.istat.it/storage/cartografia/matrici_distanze/Veneto.zip

If deemed useful, the open repository [https://github.com/openpolis/geojson-italy](https://github.com/openpolis/geojson-italy) contains a `.json` file with the geographical coordinates of the provences and comuni of Italy.


## Assignments

1. Data preparation: the csv files are originated from different sources, hence resulting in differences in the encoding and end-of-lines that have to be taken into account in the data preparation phase. Make sure each .csv file is properly interpreted.

   1.1 Ranking of visitors from foreign countries: based on the number of total visitors per each country, create a ranked plot of the first 20 countries with the most visitors
   
   1.2 Ranking of Italian visitors by province, weighted by the number of inhabitants: based on the number of total visitors per Italian province, create a ranked plot of the first 20 provinces with the most visitors taking into account the number of inhabitants.


2. Study of the visitors' fluxes: you are asked to provide indications on how to invest resources to improve the mobility towards Padova. Consider the three main directions of visitors and commuters getting to Padova through the main highways (from south, A13 towards Bologna-Roma; from west, A4 towards Milano-Torino; from north-east, A4 towards Venice-Trieste). Evaluate which of the three directions has to be prioritized.

   2.1 Consider a simplified case involving only the mid-range mobility, based on the number of visitors/commuters from the nearby regions only
   
   2.2 Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week. Use the data available to provide what you believe is the best possible answer.


3. Plot the distribution of the number of visitors by the distance of the province of origin. Determine which kind of function should be used to describe the distribution.

   3.1 Assuming an analytic form can be used to describe the trend, create a regression or a fit to estimate the expected number of visitors by the distance of the province of origin and the corresponding uncertainties. Illustrate the difference between the resulting regression with respect to the numbers provided by the Vodafone monitoring, and highlight the five most striking discrepancies from the expectations.

# Imports

In [10]:
import chardet #Used to detect the encoding of the CSV files
import codecs  #Used to read the CSV UTF-16
import io      #Used to write the CSV ISO-8859-1
import pandas as pd #Used to store data into dataframes
import matplotlib.pyplot as plt #Used to represent data
import numpy as np #Used to rename the column of dataframes

# 1 Data preparation

In [11]:
#list of csv files
filename_codici_istat_comuni = "data\codici_istat_comune.csv"
filename_codici_istat_provincia = "data\codici_istat_provincia.csv"
filename_codici_nazioni = "data\codici_nazioni.csv"
filename_day_od = "data\day_od.csv"
filename_distinct_user_day = "data\distinct_users_day.csv"
filename_distance_to_pd = "data\R05_PD.csv"
filename_distance_to_pd_txt = "data\R05_PD.txt"

#Creating a list to boost performances of the loops
filenames = [filename_codici_istat_comuni, filename_codici_istat_provincia, filename_codici_nazioni,
             filename_day_od, filename_distinct_user_day, filename_distance_to_pd]

In [12]:
# #Function which returns the encoding of each csv file
def check_encoding(file):
    #Read the file
    with open(file, 'rb') as f:
        #Detect the encoding
        result = chardet.detect(f.read())

    #Return a list of the encodings
    return result['encoding']

In [13]:
# #Function wich converts the UTF-16 encoded files into ISO-8859-1 encoded files
def encoding_converter(files):
    for file in files:
        #Saving the encoding of each file
        encodings = check_encoding(file)

        # If the encoding is different to ISO-8859-1 it has to be converted
        if encodings == 'ascii' :
            # Open the file and saving the content
            with codecs.open(file, 'r', 'ascii') as f:
                data = f.read()

            # Overwrite the file with a new encoding
            with io.open(file, 'w', encoding='utf-8') as f:
                f.write(data)

        if encodings == 'utf-16':
            # Open the file and saving the content
            with codecs.open(file, 'r', 'utf-16') as f:
                data = f.read()

            # Overwrite the file with a new encoding
            with io.open(file, 'w', encoding='latin1') as f:
                f.write(data)

### 1.1 Ranking of visitors from foreign countries

### 1.2 Ranking of visitors from Italy

# 2 Study of the visitors' fluxes

Praticamente bisogna controllare tutti i caselli autostradali delle 3 autostrade
Sommare tutti i flows attraverso i comuni delle 3 autostrade
Potrei plottare in base al casello, per ogni autostrada
I codici delle persone intendono l'origine delle persone
In realtà è una semplificazione perchè immagino che tutti quelli del nordest abbiano preso l'autostrada A4. Farò così:
Suddivido in 4 zone: 
1. NordEst (A4 To-Mi) ad Est di Padova
2. NordOvest (A4 Ts-Ve)
3. La parte sinistra dell'Italia prende l'A1 che poi diventa A13
4. La parte destra dell'Italia prende direttamente l'A13 (Emilia Marche Abruzzo Molise Puglia)

In [14]:
def toll_booths(highway = ""):   
    if highway == "A4To-Mi" :
        data_tolls = pd.DataFrame({'COMUNE':["Torino", "Borgo d'Ale", "Santhià", "Carisio", "Balocco", "Greggio", "Biandrate", "Novara",
            "Mesero", "Arluno", "Rho", "Milano", "Monza", "Agrate", "Cavenago", "Trezzo", "Capriate", "Dalmine",
            "Bergamo", "Seriate", "Grumello", "Ponte Oglio", "Palazzolo", "Rovato", "Ospitaletto", "Castegnato",
            "Brescia", "Desenzano", "Sirmione", "Peschiera", "Sommacampagna", "Verona", "Soave", "Montebello",
            "Montecchio", "Vicenza", "Grisignano"]})
        return data_tolls
        
    elif highway == "A4Ts-Ve":
        data_tolls = pd.DataFrame({'COMUNE':["Spinea", "Martellago", "Preganziol", "Meolo", "San Donà di Piave", "Cessalto",
            "San Stino di Livenza", "Latisana", "San Giorgio di Nogaro", "Palmanova", "Redipuglia", "Trieste"]})
        return data_tolls

    elif highway == "A1Ro-Bo":
        data_tolls = pd.DataFrame({'COMUNE':["Sasso Marconi", "Rioveggio", "Pian del Voglio", "Roncobilaccio", "Barberino di Mugello",
            "Calenzano", "Firenze", "Incisa", "Valdarno", "Arezzo", "Monte San Savino", "Valdichiana", "Chiusi",
            "Fabro", "Orvieto", "Attigliano", "Orte", "Magliano Sabina", "Ponzano Romano", "Guidonia Montecelio",
            "Valmontone", "Colleferro", "Anagni"]})
        return data_tolls

    elif highway == "A13Bo-Pd":
        data_tolls = pd.DataFrame({'COMUNE':["Bologna", "Altedo", "Ferrara", "Occhiobello", "Rovigo", "Boara", "Monselice", "Terme Euganee"]})
        return data_tolls
    
    elif highway == "Vi-Ve": #Da fare
        data_tolls = pd.DataFrame({'COMUNE':["Bologna", "Altedo", "Ferrara", "Occhiobello", "Rovigo", "Boara", "Monselice", "Terme Euganee"]})
        return data_tolls
    
    elif highway == "Bl-Tv-Ve": #Da fare
        data_tolls = pd.DataFrame({'COMUNE':["Bologna", "Altedo", "Ferrara", "Occhiobello", "Rovigo", "Boara", "Monselice", "Terme Euganee"]})
        return data_tolls
        
    #Da fare
    data_tolls = pd.DataFrame({'COMUNE':["Bologna", "Altedo", "Ferrara", "Occhiobello", "Rovigo", "Boara", "Monselice", "Terme Euganee"]})
    return data_tolls

In [68]:
def possible_highway(data):

    #Ignoro Padova perchè non prendono l'autostrada
    data_no = data.loc[(data['REGION'] == "Lombardia") | (data['REGION'] == "Trentino-Alta Adige") | (data['REGION'] == "Valle d'Aosta")
                       | (data['REGION'] == "Piemonte") | (data['REGION'] == "Liguria")]
    data_ne = data.loc[(data['REGION'] == "Friuli-Venezia Giulia")]
    data_so = data.loc[(data['REGION'] == "Emilia Romagna") | (data['REGION'] == "Marche") | (data['REGION'] == "Abruzzo")
                       | (data['REGION'] == "Molise") | (data['REGION'] == "Puglia")]
    data_se = data.loc[(data['REGION'] == "Toscana") | (data['REGION'] == "Umbria") | (data['REGION'] == "Lazio")
                       | (data['REGION'] == "Campania") | (data['REGION'] == "Basilicata") | (data['REGION'] == "Calabria")]
    data_ve = data.loc[(data['REGION'] == "VENETO") & ((data['COD_PRO'] == 24) | (data['COD_PRO'] == 23))] #Vicenza e Verona
    data_vo = data.loc[(data['REGION'] == "VENETO") & ((data['COD_PRO'] == 25)| (data['COD_PRO'] == 26) | (data['COD_PRO'] == 27))] #Belluno, Treviso e Venezia
    data_ro = data.loc[(data['REGION'] == "VENETO") & (data['COD_PRO'] == 29)] #Rovigo
                       
    data_groups  = [data_no, data_ne, data_so, data_se, data_ve, data_vo, data_ro]
    list_highways = ["A4To-Mi", "A4Ts-Ve", "A13Bo-Pd", "A1Ro-Bo", "Vi-Ve", "Bl-Ve-Tv", "Ro"] 

    for x in range(0,len(data_groups)):
        # CALCOLO DEI FLUSSI DEI COMUNI CON CASELLO
        comuni_toll = toll_booths(list_highways[x])
        #Controllo corrispondenze, se trovo corrispondenze calcolo la somma
        data_comuni_toll = pd.merge(data_groups[x], comuni_toll, on="COMUNE")

        # CALCOLO DEI FLUSSI DELLE PROVINCE CON CASELLO
        #Altrimenti controllo la provincia
        comuni_codes = pd.read_csv(filename_codici_istat_comuni, encoding='latin1', usecols=['COMUNE', 'PRO_COM', 'COD_PRO', 'COMUNE_CAPOLUOGO'])
        comuni_toll = pd.merge(comuni_toll, comuni_codes, on="COMUNE")
        # Subtraction between two dataframe
        data_without_comuni_toll = data_groups[x].merge(data_comuni_toll, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
        # Prendo le province dei comuni con i caselli
        data_province_toll = data_without_comuni_toll.drop(['COMUNE','PRO_COM'], axis= 1)
        data_province_toll = pd.merge(data_province_toll, comuni_toll, on='COD_PRO')
        data_province_toll = data_province_toll.groupby(['COD_PRO'])[['VISITORS']].sum()

        # Unire i due DataFrame in base alla colonna 'COD_PRO'
        data_com_pro_toll = pd.merge(data_province_toll, data_comuni_toll, suffixes=('_df1', '_df2'), on='COD_PRO', how='outer')

        # Sommare i valori della colonna 'visitors' dei due DataFrame
        data_com_pro_toll['VISITORS'] = data_com_pro_toll['VISITORS_df1'].fillna(0) + data_com_pro_toll['VISITORS_df2'].fillna(0)
        data_com_pro_toll = data_com_pro_toll.drop(['VISITORS_df1','VISITORS_df2'], axis= 1)

        data_com_pro_toll = data_com_pro_toll.dropna()

        # CALCOLO DEI FLUSSI PER USER CHE NON HANNO UN CASELLO IN PROVINCIA -> DISTRIBUISCO UNIFORMEMENTE PER I CASELLI
        # Prendo tutte le province del data_groups[x] -> Ci tolgo quelle con il casello
        province = set(data_groups[x]['COD_PRO'].unique())
        province_with_toll = set(data_com_pro_toll['COD_PRO'].unique())
        province_without_toll = pd.DataFrame({'COD_PRO':list(province - province_with_toll)})
        
        data_region_toll = pd.merge(data_groups[x], province_without_toll, on="COD_PRO")
        
        # Sommo il flusso delle province
        data_region_toll = data_region_toll.groupby('COD_PRO')[['VISITORS']].sum()
        # Uniformo
        flow = (data_region_toll['VISITORS'].sum() / len(data_com_pro_toll)).round()
        # Sommo
        data_com_pro_toll['VISITORS'] = data_com_pro_toll['VISITORS'] + flow
        
    return data_groups

In [69]:
visitors_fluxes(filename_distinct_user_day)

C:\Users\ecapo\AppData\Local\Temp\ipykernel_55012\272080303.py:55: RuntimeWarning: invalid value encountered in scalar divide
  flow = (data_region_toll['VISITORS'].sum() / len(data_com_pro_toll)).round()
C:\Users\ecapo\AppData\Local\Temp\ipykernel_55012\272080303.py:55: RuntimeWarning: invalid value encountered in scalar divide
  flow = (data_region_toll['VISITORS'].sum() / len(data_com_pro_toll)).round()
C:\Users\ecapo\AppData\Local\Temp\ipykernel_55012\272080303.py:55: RuntimeWarning: invalid value encountered in scalar divide
  flow = (data_region_toll['VISITORS'].sum() / len(data_com_pro_toll)).round()


[       PRO_COM  VISITORS  COD_PRO              COMUNE  COD_REG     REGION
 0       1002.0        36        1             Airasca        1   Piemonte
 1       1008.0       268        1           Alpignano        1   Piemonte
 2       1013.0       220        1           Avigliana        1   Piemonte
 3       1024.0        36        1            Beinasco        1   Piemonte
 4       1028.0       344        1    Borgaro Torinese        1   Piemonte
 ...        ...       ...      ...                 ...      ...        ...
 2260  108048.0       308      108      Verano Brianza        3  Lombardia
 2261  108049.0       568      108          Villasanta        3  Lombardia
 2262  108050.0       904      108           Vimercate        3  Lombardia
 2263  108053.0       496      108      Cornate d'Adda        3  Lombardia
 2264  108054.0      1116      108  Lentate sul Seveso        3  Lombardia
 
 [727 rows x 6 columns],
       PRO_COM  VISITORS  COD_PRO                   COMUNE  COD_REG  \
 1

In [17]:
def add_region(data):
    regioni = {
        'Abruzzo': 13,
        'Basilicata': 17,
        'Calabria': 18,
        'Campania': 15,
        'Emilia Romagna': 8,
        'Friuli-Venezia Giulia': 6,
        'Lazio': 12,
        'Liguria': 7,
        'Lombardia': 3,
        'Marche': 11,
        'Molise': 14,
        'Piemonte': 1,
        'Puglia': 16,
        'Sardegna': 20,
        'Sicilia': 19,
        'Toscana': 9,
        'Trentino-Alto Adige': 4,
        'Umbria': 10,
        'Valle d\'Aosta': 2,
        'Veneto': 5}
    
    # Inizializza una lista per memorizzare i nomi delle regioni
    region_names = []
    
    # Itera attraverso ogni riga del DataFrame
    for index, row in data.iterrows():
        # Ottieni il codice della regione dalla colonna 'COD_REG'
        cod_reg = row['COD_REG']
        
        # Cerca il nome della regione corrispondente al codice nella colonna 'COD_REG'
        for region_name, region_code in regioni.items():
            if cod_reg == region_code:
                # Aggiungi il nome della regione alla lista
                region_names.append(region_name)
                break
        else:
            # Se il codice della regione non è presente nel dizionario 'regioni', aggiungi None
            region_names.append(None)
    
    # Aggiungi la lista dei nomi delle regioni come nuova colonna 'REGIONE' nel DataFrame
    data['REGION'] = region_names

    return data

In [18]:
def visitors_fluxes(file_customers, dow_study=False):
    '''
    Consider the three main directions:
    A13 Roma-Bologna
    A4 Torino-Milano
    A4 Trieste-Venezia
    '''
    #Consider the istat province codes
    comuni_codes = pd.read_csv(filename_codici_istat_comuni, encoding='latin1', usecols=['COMUNE', 'PRO_COM', 'COD_PRO'])
    provinces_codes = pd.read_csv(filename_codici_istat_provincia, encoding='latin1', usecols=['COD_PRO', 'COD_REG'])

    codes = pd.merge(comuni_codes, provinces_codes, on='COD_PRO')

    if not dow_study:
        #Now take the data of the customers and see what's their origin and their destination
        customers_data = pd.read_csv(file_customers, encoding='latin1', usecols = ['VISITORS', 'PRO_COM'])
    else:
        customers_data = pd.read_csv(file_customers, encoding='latin1', usecols = ['VISITORS', 'DOW', 'PRO_COM'])

    customers_data.dropna(axis=0, inplace=True)

    user_regions = add_region(codes)
    
    #Join between customers_data and province_codes in this way filters the province_codes in which we're not interested
    customers_data = customers_data.groupby('PRO_COM').sum()
    
    data = pd.merge(customers_data, user_regions, on='PRO_COM')

    highways = possible_highway(data)

    return highways

In [19]:
def plot_visitors(data, range_of_mobility=""):
    #Counting the flows ov visitors from every city we're interested (both ways)
    data = data.groupby(['PROVINCIA'])[['FLOW']].sum()

    far = True

    if range_of_mobility == "nearby":
        far = False

    #Saving the informations per highway
    A13_Roma_Bologna = far * data.FLOW.loc['Roma'] + data.FLOW.loc['Bologna']
    A4_Milano_Torino = data.FLOW.loc['Milano'] + far * data.FLOW.loc['Torino']
    A4_Venezia_Trieste = data.FLOW.loc['Venezia'] + data.FLOW.loc['Trieste']

    highway = [A13_Roma_Bologna, A4_Milano_Torino, A4_Venezia_Trieste]

    #Labels for the plot
    highway_labels = ["A13 Roma-Bologna", "A4 Milano-Torino", "A4 Venezia-Trieste"]
    
    # Plotting
    plt.figure(figsize=(10, 4))
    plt.bar(highway_labels, highway, color='skyblue')
    plt.xlabel('Highways')
    plt.ylabel('Visitors'' flow')
    plt.title('Visitors'' flow per highway')
    # Set the scale to avoid the exponential notation 
    plt.ticklabel_format(style='plain', axis='y')
    plt.tight_layout()

    plt.show()

In [20]:
plot_visitors(visitors_fluxes(filename_day_od))

ValueError: Usecols do not match columns, columns expected but not found: ['VISITORS']

### 2.1.b Mid-range mobility
Mobility to/from nearby regions, which are Lombardia, Trentino Alto Adige, Friuli Venezia Giulia, Emilia Romagna.
QUESTA SOLUZIONE TIENE CONTO DELLE AUTOSTRADE

In [ ]:
plot_visitors(visitors_fluxes(filename_day_od), "nearby")

### 2.2 Week flow

Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week. Use the data available to provide what you believe is the best possible answer.

In [ ]:
def plotter(data1, data2, label):
    #Saving the informations per highway
    A13_Roma_Bologna = data1.FLOW.loc['Roma'] + data1.FLOW.loc['Bologna']
    A4_Milano_Torino = data1.FLOW.loc['Milano'] + data1.FLOW.loc['Torino']
    A4_Venezia_Trieste = data1.FLOW.loc['Venezia'] + data1.FLOW.loc['Trieste']

    highway1 = [A13_Roma_Bologna, A4_Milano_Torino, A4_Venezia_Trieste]

    #Saving the informations per highway
    A13_Roma_Bologna = data2.FLOW.loc['Roma'] + data2.FLOW.loc['Bologna']
    A4_Milano_Torino = data2.FLOW.loc['Milano'] + data2.FLOW.loc['Torino']
    A4_Venezia_Trieste = data2.FLOW.loc['Venezia'] + data2.FLOW.loc['Trieste']

    highway2 = [A13_Roma_Bologna, A4_Milano_Torino, A4_Venezia_Trieste]

    #Labels for the plot
    highway_labels = ["A13 Roma-Bologna", "A4 Milano-Torino", "A4 Venezia-Trieste"]
    
    # Plotting
    plt.figure(figsize=(10, 4))
    '''
    plt.bar(highway_labels, highway, color='skyblue')
    plt.xlabel(label)
    plt.ylabel('Visitors'' flow')
    plt.title('Visitors'' flow in ' + label)
    # Set the scale to avoid the exponential notation 
    plt.ticklabel_format(style='plain', axis='y')
    plt.tight_layout()
    '''
    # Plot dei valori
    plt.plot(highway_labels, highway1, marker='o')  # 'marker' specifica il tipo di marker per i punti
    plt.plot(highway_labels, highway2, marker='x')  # 'marker' specifica il tipo di marker per i punti
    plt.xlabel('X')  # Etichetta asse x
    plt.ylabel('Y')  # Etichetta asse y
    plt.title('Plot tipo funzione dei valori di un DataFrame')  # Titolo del grafico
    plt.grid(False)  # Abilita la griglia

    plt.show()
    #Fare istogramma grouped bar

In [ ]:
'''
Seleziono i turisti per le tre direzioni, poi li suddivido in weekend e working days
'''
def dow_visitors_fluxes(data):
    data_weekend = data.loc[(data['DOW'] == "Domenica") | (data['DOW'] == "Sabato")]

    data_working_day = data.loc[~data.index.isin(data_weekend.index)]

    # #Join between customers_data and province_codes in this way filters the province_codes in which we're not interested
    # data_weekend = pd.merge(weekend_flow, data, on='COD_PRO')
    # data_working_day = pd.merge(working_day_flow, data, on='COD_PRO')

    #Counting the flows ov visitors from every city we're interested (both ways)
    data_weekend = data_weekend.groupby(['PROVINCIA'])[['FLOW']].sum()
    data_working_day = data_working_day.groupby(['PROVINCIA'])[['FLOW']].sum()

    plotter(data_weekend, data_working_day, "Weekend")
    # plotter(data_working_day, "Working ")

In [ ]:
dow_visitors_fluxes(visitors_fluxes(filename_day_od))

# 3
Plot the distribution of the number of visitors by the distance of the province of origin. Determine which kind of function should be used to describe the distribution.

è scritto visitor quindi tolgo i non visitor

In [ ]:
def data_visitors_distance(province_data):
    distances_data = pd.read_csv(province_data, sep="\t", encoding="UTF-8", usecols=['DEST_PROCOM', 'KM_TOT'])

    distances_data.rename(columns={'DEST_PROCOM': 'PRO_COM'}, inplace=True)

    distances_data['KM_TOT'] = distances_data['KM_TOT'].str.replace(',','.')
    distances_data['KM_TOT'] = pd.to_numeric(distances_data['KM_TOT'])

    distances_data = distances_data.groupby(['PRO_COM'])[['KM_TOT']].mean()

    customers_data = pd.read_csv(filename_distinct_user_day, encoding="latin1", usecols=['PRO_COM','VISITORS','CUST_CLASS'])

    # Escludo Padova
    customers_data['PRO_COM'] = np.where(customers_data['PRO_COM'] == 28060.0 , np.nan,
                                np.where(customers_data['CUST_CLASS'] != 'visitor', np.nan, customers_data['PRO_COM']))
    customers_data.dropna(axis=0, inplace=True)

    # Elimina la colonna "CUST_CLASS"
    customers_data = customers_data.drop('CUST_CLASS', axis=1)

    customers_data = customers_data.groupby(['PRO_COM'])[['VISITORS']].sum()

    data = pd.merge(distances_data, customers_data, on='PRO_COM')

    data = data.sort_values(['VISITORS'], ascending=False)

    return data

In [ ]:
data_visitors_distance(filename_distance_to_pd_txt)

In [ ]:
def plot_visitors_distance(data, zoom=1):
    # zoom serve a raggruppare le distanze (50 --> raggruppo le distanze ogni 50 km)
    # calcolo il range di raggruppamento
    period = int(data['KM_TOT'].max() / zoom)
    somma_visitatori = []
    for n_group in range(1,period+1):
        data_grouped = data[data['KM_TOT'].between((n_group-1)*zoom +1 , n_group * zoom)]
        somma_visitatori.append(data_grouped['VISITORS'].sum())
    
    data_to_plot = pd.DataFrame({'DISTANCE_TO_PD': [km * zoom for km in range(0, period)], 'VISITORS': somma_visitatori})
    # Impostare l'indice dopo la creazione del DataFrame
    data_to_plot.set_index('DISTANCE_TO_PD', inplace=True)

    # data_to_plot['KM_TOT'] = np.where(data_to_plot['KM_TOT'] == 0 , np.nan, data_to_plot['KM_TOT'])
    # data_to_plot.dropna(axis=0, inplace=True)

    # data_nearby = data.loc[(data['KM_TOT'] < 50)]

    # data_far = data.loc[~data.index.isin(data_nearby.index)]

    plt.figure(figsize=(10, 6))
    plt.hist(data_to_plot['VISITORS'], color='blue', alpha=0.7, bins=20)
    plt.xlabel('Distance from Padova (KM)')
    plt.ylabel('Number of Visitors')
    plt.title('Histogram of Distance from Padova')
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.show()
    #Da riguardare il grafico

In [ ]:
plot_visitors_distance(data_visitors_distance(filename_distance_to_pd_txt), 50)

### 3.1 
Assuming an analytic form can be used to describe the trend, create a regression or a fit to estimate the expected number of visitors by the distance of the province of origin and the corresponding uncertainties. Illustrate the difference between the resulting regression with respect to the numbers provided by the Vodafone monitoring, and highlight the five most striking discrepancies from the expectations.

In [ ]:
from sklearn.linear_model import LinearRegression

# Build linear regression model using TV and Radio as predictors
# Split data into predictors X and output Y
X = np.array(data['KM_TOT']).reshape(-1,1)
y = data['VISITORS']

# Initialise and fit model
lm = LinearRegression()
model = lm.fit(X, y)
# plot for residual error
 
   
plt.scatter(X,y,color='red')
plt.plot(X,model.predict(X),color='green')
plt.title('Simple Linear Regression')
plt.xlabel('Position Level')
plt.ylabel('Salary')
plt.show()